In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=73263db45a0a1692169f98dbb723dc51dd8fa899ef8337caf39129b8e4aa7e25
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle -v

Kaggle API 1.5.6


In [ ]:
!kaggle competitions download -c aidefensegame18011862

  0% 0.00/270k [00:00<?, ?B/s]
100% 270k/270k [00:00<00:00, 77.7MB/s]


In [ ]:
!unzip aidefensegame18011862.zip

Archive:  aidefensegame18011862.zip
  inflating: 18011862Aitrain.csv     
  inflating: 18011862aitest.csv      
  inflating: kaggle_18011862submission.csv  
  inflating: kaggle_18011862test.csv  
  inflating: kaggle_2040train.csv    


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn.functional as F  

In [ ]:
my_na_values = ['불명']
data = pd.read_csv('18011862Aitrain.csv',na_values=my_na_values)
data

,연번,성별,적발횟수,나이,알콜농도,측정일시,관할경찰서
0,1,남자,1,30.0,0.105,2020-01-01 0:03,일산동부경찰서
1,2,남자,1,57.0,0.077,2020-01-01 0:13,부산연제경찰서
2,3,남자,1,68.0,0.089,2020-01-01 0:17,부천소사경찰서
3,4,남자,1,39.0,0.133,2020-01-01 0:17,마산중부경찰서
4,5,남자,1,37.0,측정거부,2020-01-01 0:19,수원서부경찰서
...,...,...,...,...,...,...,...
16059,16060,남자,1,40.0,0.164,2020-02-29 23:49,대전유성경찰서
16060,16061,남자,1,36.0,0.185,2020-02-29 23:49,울산남부경찰서
16061,16062,여자,1,46.0,0.122,2020-02-29 23:50,이천경찰서
16062,16063,남자,1,26.0,0.097,2020-02-29 23:51,수원남부경찰서


In [ ]:
data['관할경찰서'][data.관할경찰서.str.contains('광주')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('서울')] = '0'
data['관할경찰서'][data.관할경찰서.str.contains('부산')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('대구')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('인천')] = '2'
data['관할경찰서'][data.관할경찰서.str.contains('대전')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('울산')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('수원')|data.관할경찰서.str.contains('일산')|data.관할경찰서.str.contains('성남')|data.관할경찰서.str.contains('용인')|data.관할경찰서.str.contains('안양')|data.관할경찰서.str.contains('안산')|data.관할경찰서.str.contains('과천')|data.관할경찰서.str.contains('광명')|data.관할경찰서.str.contains('군포')|data.관할경찰서.str.contains('부천')|data.관할경찰서.str.contains('시흥')|data.관할경찰서.str.contains('김포')|data.관할경찰서.str.contains('안성')|data.관할경찰서.str.contains('오산')|data.관할경찰서.str.contains('의왕')|data.관할경찰서.str.contains('이천')|data.관할경찰서.str.contains('평택')|data.관할경찰서.str.contains('하남')|data.관할경찰서.str.contains('화성')|data.관할경찰서.str.contains('여주')|data.관할경찰서.str.contains('양평')|data.관할경찰서.str.contains('고양')|data.관할경찰서.str.contains('구리')|data.관할경찰서.str.contains('남양주')|data.관할경찰서.str.contains('동두천')|data.관할경찰서.str.contains('양주')|data.관할경찰서.str.contains('의정부')|data.관할경찰서.str.contains('파주')|data.관할경찰서.str.contains('포천')|data.관할경찰서.str.contains('연천')|data.관할경찰서.str.contains('가평')|data.관할경찰서.str.contains('분당')] = '1'
data['관할경찰서'][data.관할경찰서.str.contains('춘천')|data.관할경찰서.str.contains('원주')|data.관할경찰서.str.contains('강릉')|data.관할경찰서.str.contains('동해')|data.관할경찰서.str.contains('태백')|data.관할경찰서.str.contains('속초')|data.관할경찰서.str.contains('삼척')|data.관할경찰서.str.contains('홍천')|data.관할경찰서.str.contains('횡성')|data.관할경찰서.str.contains('영월')|data.관할경찰서.str.contains('평창')|data.관할경찰서.str.contains('정선')|data.관할경찰서.str.contains('철원')|data.관할경찰서.str.contains('화천')|data.관할경찰서.str.contains('양구')|data.관할경찰서.str.contains('인제')|data.관할경찰서.str.contains('고성')|data.관할경찰서.str.contains('양양')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('청주')|data.관할경찰서.str.contains('상당')|data.관할경찰서.str.contains('서원')|data.관할경찰서.str.contains('흥덕')|data.관할경찰서.str.contains('청원')|data.관할경찰서.str.contains('충주')|data.관할경찰서.str.contains('제천')|data.관할경찰서.str.contains('보은')|data.관할경찰서.str.contains('옥천')|data.관할경찰서.str.contains('영동')|data.관할경찰서.str.contains('증평')|data.관할경찰서.str.contains('진천')|data.관할경찰서.str.contains('괴산')|data.관할경찰서.str.contains('음성')|data.관할경찰서.str.contains('단양')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('천안')|data.관할경찰서.str.contains('공주')|data.관할경찰서.str.contains('보령')|data.관할경찰서.str.contains('아산')|data.관할경찰서.str.contains('서산')|data.관할경찰서.str.contains('논산')|data.관할경찰서.str.contains('계룡')|data.관할경찰서.str.contains('당진')|data.관할경찰서.str.contains('금산')|data.관할경찰서.str.contains('부여')|data.관할경찰서.str.contains('서천')|data.관할경찰서.str.contains('청양')|data.관할경찰서.str.contains('홍성')|data.관할경찰서.str.contains('예산')|data.관할경찰서.str.contains('태안')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('전주')|data.관할경찰서.str.contains('군산')|data.관할경찰서.str.contains('익산')|data.관할경찰서.str.contains('정읍')|data.관할경찰서.str.contains('정읍')|data.관할경찰서.str.contains('남원')|data.관할경찰서.str.contains('김제')|data.관할경찰서.str.contains('완주')|data.관할경찰서.str.contains('진안')|data.관할경찰서.str.contains('무주')|data.관할경찰서.str.contains('장수')|data.관할경찰서.str.contains('임실')|data.관할경찰서.str.contains('순창')|data.관할경찰서.str.contains('고창')|data.관할경찰서.str.contains('부안')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('목포')|data.관할경찰서.str.contains('여수')|data.관할경찰서.str.contains('순천')|data.관할경찰서.str.contains('나주')|data.관할경찰서.str.contains('광양')|data.관할경찰서.str.contains('담양')|data.관할경찰서.str.contains('곡성')|data.관할경찰서.str.contains('구례')|data.관할경찰서.str.contains('고흥')|data.관할경찰서.str.contains('보성')|data.관할경찰서.str.contains('화순')|data.관할경찰서.str.contains('장흥')|data.관할경찰서.str.contains('강진')|data.관할경찰서.str.contains('해남')|data.관할경찰서.str.contains('영암')|data.관할경찰서.str.contains('무안')|data.관할경찰서.str.contains('함평')|data.관할경찰서.str.contains('영광')|data.관할경찰서.str.contains('장성')|data.관할경찰서.str.contains('완도')|data.관할경찰서.str.contains('진도')|data.관할경찰서.str.contains('신안')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('포항')|data.관할경찰서.str.contains('경주')|data.관할경찰서.str.contains('김천')|data.관할경찰서.str.contains('안동')|data.관할경찰서.str.contains('구미')|data.관할경찰서.str.contains('영주')|data.관할경찰서.str.contains('영천')|data.관할경찰서.str.contains('영천')|data.관할경찰서.str.contains('상주')|data.관할경찰서.str.contains('문경')|data.관할경찰서.str.contains('경산')|data.관할경찰서.str.contains('군위')|data.관할경찰서.str.contains('의성')|data.관할경찰서.str.contains('청송')|data.관할경찰서.str.contains('영양')|data.관할경찰서.str.contains('영덕')|data.관할경찰서.str.contains('청도')|data.관할경찰서.str.contains('고령')|data.관할경찰서.str.contains('성주')|data.관할경찰서.str.contains('칠곡')|data.관할경찰서.str.contains('예천')|data.관할경찰서.str.contains('봉화')|data.관할경찰서.str.contains('울진')|data.관할경찰서.str.contains('울릉')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('창원')|data.관할경찰서.str.contains('진주')|data.관할경찰서.str.contains('통영')|data.관할경찰서.str.contains('사천')|data.관할경찰서.str.contains('김해')|data.관할경찰서.str.contains('밀양')|data.관할경찰서.str.contains('거제')|data.관할경찰서.str.contains('양산')|data.관할경찰서.str.contains('의령')|data.관할경찰서.str.contains('함안')|data.관할경찰서.str.contains('창녕')|data.관할경찰서.str.contains('고성')|data.관할경찰서.str.contains('남해')|data.관할경찰서.str.contains('하동')|data.관할경찰서.str.contains('산청')|data.관할경찰서.str.contains('함양')|data.관할경찰서.str.contains('거창')|data.관할경찰서.str.contains('합천')|data.관할경찰서.str.contains('마산')|data.관할경찰서.str.contains('진해')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('제주')|data.관할경찰서.str.contains('서귀포')] = 'hi'
data['관할경찰서'][data.관할경찰서.str.contains('반곡')|data.관할경찰서.str.contains('소담')|data.관할경찰서.str.contains('보람')|data.관할경찰서.str.contains('대평')|data.관할경찰서.str.contains('가람')|data.관할경찰서.str.contains('한솔')|data.관할경찰서.str.contains('나성')|data.관할경찰서.str.contains('새롬')|data.관할경찰서.str.contains('다정')|data.관할경찰서.str.contains('어진')|data.관할경찰서.str.contains('종촌')|data.관할경찰서.str.contains('고운')|data.관할경찰서.str.contains('아름')|data.관할경찰서.str.contains('도담')|data.관할경찰서.str.contains('조치원')|data.관할경찰서.str.contains('연기')|data.관할경찰서.str.contains('연동')|data.관할경찰서.str.contains('부강')|data.관할경찰서.str.contains('금남')|data.관할경찰서.str.contains('장군')|data.관할경찰서.str.contains('연서')|data.관할경찰서.str.contains('전의')|data.관할경찰서.str.contains('전동')|data.관할경찰서.str.contains('소정')|data.관할경찰서.str.contains('세종')] = 'hi'

replace_values = {'남자' : 0,'여자' : 1} #남자:0,여자:1
data = data.replace({"성별": replace_values})

data1 = data[data['관할경찰서'] == 'hi'].index
data = data.drop(data1) #광주행 다 삭제

data = data.dropna(axis=0) #결측값 삭제

data = data[['성별','적발횟수','나이','측정일시','관할경찰서']]
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

,성별,적발횟수,나이,측정일시,관할경찰서
0,0.0,1,30.0,2020-01-01 0:03,1
2,0.0,1,68.0,2020-01-01 0:17,1
4,0.0,1,37.0,2020-01-01 0:19,1
10,0.0,1,27.0,2020-01-01 0:35,1
13,0.0,1,46.0,2020-01-01 0:41,2
...,...,...,...,...,...
16055,0.0,1,49.0,2020-02-29 23:40,1
16058,0.0,1,47.0,2020-02-29 23:45,0
16061,1.0,1,46.0,2020-02-29 23:50,1
16062,0.0,1,26.0,2020-02-29 23:51,1


In [ ]:
m_tmp = data[['측정일시']]
d_tmp = data[['측정일시']]
d_tmp['측정일시'] = data['측정일시'].str.slice(8,10) #일
m_tmp['측정일시'] = data['측정일시'].str.slice(5,7) # m:월
replace_values = {'01' :13,'02':14} 
m_tmp = m_tmp.replace({"측정일시": replace_values})
m = m_tmp['측정일시']
d = data['측정일시'].str.slice(8,10) 
d = d.apply(pd.to_numeric)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
days = {1:'SUN' , 2:'MON', 
        3:'TUE', 4:'WED', 
        5:'THU', 6:'FRI', 0:'SAT'}
# 첼러의 공식
var = ((m+1)*13)/5
var = var.astype('int')
h = (d +  var + 19 + int(19/4) + int(20/4) - 2*20 ) % 7
h 

0        4
2        4
4        4
10       4
13       4
        ..
16055    0
16058    0
16061    0
16062    0
16063    0
Name: 측정일시, Length: 7113, dtype: int64

In [ ]:
data['측정일시'] = h
data

,성별,적발횟수,나이,측정일시,관할경찰서
0,0.0,1,30.0,4,1
2,0.0,1,68.0,4,1
4,0.0,1,37.0,4,1
10,0.0,1,27.0,4,1
13,0.0,1,46.0,4,2
...,...,...,...,...,...
16055,0.0,1,49.0,0,1
16058,0.0,1,47.0,0,0
16061,1.0,1,46.0,0,1
16062,0.0,1,26.0,0,1


In [ ]:
newdata = data[['나이']]
newdata

,나이
0,30.0
2,68.0
4,37.0
10,27.0
13,46.0
...,...
16055,49.0
16058,47.0
16061,46.0
16062,26.0


In [ ]:
newdata[(newdata['나이'] < 40)] = 0
newdata[(newdata['나이'] >= 40)] = 1
newdata

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2957: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,나이
0,0.0
2,1.0
4,0.0
10,0.0
13,1.0
...,...
16055,1.0
16058,1.0
16061,1.0
16062,0.0


In [ ]:
data = data[['성별','적발횟수','측정일시','관할경찰서']]
data = pd.concat([data,newdata], axis=1) #열합치기
data

,성별,적발횟수,측정일시,관할경찰서,나이
0,0.0,1,4,1,0.0
2,0.0,1,4,1,1.0
4,0.0,1,4,1,0.0
10,0.0,1,4,1,0.0
13,0.0,1,4,2,1.0
...,...,...,...,...,...
16055,0.0,1,0,1,1.0
16058,0.0,1,0,0,1.0
16061,1.0,1,0,1,1.0
16062,0.0,1,0,1,0.0


In [ ]:
data = data.astype('float32')
data = data.to_numpy()
data

array([[0., 1., 4., 1., 0.],
       [0., 1., 4., 1., 1.],
       [0., 1., 4., 1., 0.],
       ...,
       [1., 1., 0., 1., 1.],
       [0., 1., 0., 1., 0.],
       [1., 1., 0., 1., 1.]], dtype=float32)

In [ ]:
x_data = data[: , 0:-1]
y_data = data[: , -1]

x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

In [ ]:
import torch.nn.functional as F  

nb_class = 2
nb_data = len(y_train)

W = torch.zeros((4, nb_class), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1)
nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 0.693146
Epoch 1000/10000 Cost: 0.690121
Epoch 2000/10000 Cost: 0.690121
Epoch 3000/10000 Cost: 0.690121
Epoch 4000/10000 Cost: 0.690121
Epoch 5000/10000 Cost: 0.690121
Epoch 6000/10000 Cost: 0.690121
Epoch 7000/10000 Cost: 0.690121
Epoch 8000/10000 Cost: 0.690121
Epoch 9000/10000 Cost: 0.690121
Epoch 10000/10000 Cost: 0.690121


In [ ]:
hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 
predict = torch.argmax(hypothesis, dim=1)

#print(hypothesis)
#print(predict)
#print(y_train)

correct_prediction = predict.float() == y_train
#print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('정확도:{:2.2f}%'.format(accuracy * 100))

정확도:52.31%


In [ ]:
testdata = pd.read_csv('kaggle_18011862test.csv')
testdata

,성별,적발횟수,측정일시,관할경찰서
0,0.0,1,1,1
1,0.0,1,1,1
2,0.0,1,1,1
3,0.0,1,1,1
4,1.0,1,1,0
...,...,...,...,...
2714,0.0,1,2,1
2715,0.0,1,2,1
2716,0.0,1,2,1
2717,0.0,1,2,2


In [ ]:
testdata = testdata.astype('float32')
testdata = testdata.to_numpy()

In [ ]:
x_data = testdata[: , :]
x_train = torch.FloatTensor(x_data)

In [ ]:
hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 
predict = torch.argmax(hypothesis, dim=1)

In [ ]:
predict = predict.detach().numpy().reshape(-1,1)
#import pbd;pbd.set_trace()
id=np.array([i for i in range(2719)]).reshape(-1,1)
result=np.hstack([id,predict])
df=pd.DataFrame(result,columns=["ID","Label"])
#df.to_csv("result.csv",index=False,header=True)

In [ ]:
df['Label'] = df['Label'].astype('float32')

In [ ]:
df.to_csv("baseline.csv",index=False,header=True)

In [ ]:
!kaggle competitions submit -c aidefensegame18011862 -f baseline.csv -m "18011862_joyoonsik"

100% 22.8k/22.8k [00:00<00:00, 118kB/s]
Successfully submitted to AIDefenseGame18011862